In [1]:
import sys
sys.path.append('/home/cfusting/Dropbox/arctic-browning')
sys.path.append('/home/cfusting/gp_mecl')

In [2]:
import importlib
EXPERMINENT_NAME = "penguin"
experiment = importlib.import_module("experiments." + EXPERMINENT_NAME)

In [3]:
from utilities import lib
TRAINING = "/home/cfusting/design_matrices/training_matrix.hdf"
predictors, response = lib.get_predictors_and_response(TRAINING)
lst_days, snow_days = lib.get_lst_and_snow_days(TRAINING)
pset = experiment.get_pset(predictors.shape[1], lst_days, snow_days)

In [4]:
import cachetools
from deap import creator, base
from gp.algorithms import afpo
creator.create("ErrorSizeComplexity", base.Fitness, weights=(-1.0, -1.0, -1.0))
validate_toolbox = experiment.get_validation_toolbox(predictors, response, pset,
                                                     size_measure=afpo.evaluate_fitness_size_complexity,
                                                     expression_dict=cachetools.LRUCache(maxsize=100),
                                                     fitness_class=creator.ErrorSizeComplexity)

In [5]:
import glob
RESULTS_PATH = "/home/cfusting/symbolic_results"
SEEDS = ['17166', '17785', '246', '25960', '26777', '29676', '423', '8761']
from ndvi import gp_processing_tools
pareto_files = glob.glob(RESULTS_PATH + "/pareto_*_po_{}_*.log".format(EXPERMINENT_NAME))
filtered_files = []
for s in SEEDS:
    filtered_files.extend([x for x in pareto_files if s in x])
all_inds = dict()
for f in filtered_files:
    last_front = gp_processing_tools.get_last_pareto_front(f)
    all_inds[f] = []
    for ind_tuple in last_front:
        tree_string = ind_tuple[1]
        pareto_ind = creator.Individual.from_string(tree_string, pset)
        pareto_ind = creator.Individual(pareto_ind)
        all_inds[f].append(pareto_ind)

/home/cfusting/symbolic_results/pareto_afsc_po_penguin_17166.log
/home/cfusting/symbolic_results/pareto_afsc_po_penguin_17785.log
/home/cfusting/symbolic_results/pareto_afsc_po_penguin_246.log
/home/cfusting/symbolic_results/pareto_afsc_po_penguin_25960.log
/home/cfusting/symbolic_results/pareto_afsc_po_penguin_26777.log
/home/cfusting/symbolic_results/pareto_afsc_po_penguin_29676.log
/home/cfusting/symbolic_results/pareto_afsc_po_penguin_423.log
/home/cfusting/symbolic_results/pareto_afsc_po_penguin_8761.log


/home/cfusting/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [42]:
a = all_inds.keys()[0]

In [43]:
all_inds[a]

[[<deap.gp.Primitive at 0x7fc2f67e1e68>,
 [<deap.gp.Primitive at 0x7fc2f67e1f18>, <deap.gp.Terminal at 0x7fc2e5550780>],

In [10]:
from gp.parametrized import simple_parametrized_terminals as sp
import pygraphviz as pgv
i = 0
for key in all_inds.keys():
    for ind in all_inds[key]:
        nodes, edges, labels = sp.graph(ind)
        g = pgv.AGraph()
        g.add_nodes_from(nodes)
        g.add_edges_from(edges)
        g.layout(prog="dot")
        for i in nodes:
            n = g.get_node(i)
            n.attr["label"] = labels[i]
        g.draw(key + "_tree_" + str(i) + ".eps")
        i += 1